# Import Libraries

In [ ]:
!pip install -U -q ../input/resnest/resnest-0.0.5-py3-none-any.whl

In [ ]:
import os
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from albumentations import (
    HorizontalFlip, VerticalFlip, Transpose, RandomResizedCrop, Compose, Normalize, ShiftScaleRotate, CenterCrop, Resize, RandomResizedCrop
)
from albumentations.pytorch import ToTensorV2
from glob import glob
from resnest.torch import resnest50, resnest50_fast_4s2x40d
from torch import nn
from torch.utils.data import Dataset
from tqdm import tqdm

In [ ]:
import sys

sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [ ]:
import timm

# Configuration

In [ ]:
CFG = {
    'seed': 1337,
    'img_size': 512,
    'bs': 32,
    'num_workers': 4,
    'tta': 4,
}

# Load Data

In [ ]:
submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
submission.head()

# Helper Functions

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
def get_img(path):
    return cv2.imread(path)[:, :, ::-1]

In [ ]:
seed_everything(CFG['seed'])

# Dataset

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, df, data_root, transforms=None):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index: int):
        img  = get_img(f"{self.data_root}/{self.df.iloc[index]['image_id']}")
        if self.transforms:
            img = self.transforms(image=img)['image']
        return img

# Augmentations

In [ ]:
def get_inference_transforms():
    return Compose([
        # RandomResizedCrop(CFG['img_size'], CFG['img_size']),
        # Resize(CFG['img_size'], CFG['img_size']),
        CenterCrop(CFG['img_size'], CFG['img_size']),
        # Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        # HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        # RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)

# Model

In [ ]:
class CassvaClassifierV1(nn.Module):
    def __init__(self, n_classes: int = 5, dropout: float = .5):
        super().__init__()
        self.backbone = resnest50_fast_4s2x40d(pretrained=False)

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(p=dropout)
        self.emb_size: int = 2048

        self.classifier = nn.Linear(self.emb_size, n_classes)

    def cnn_feature_extractor(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x1 = self.backbone.layer1(x)
        x2 = self.backbone.layer2(x1)
        x3 = self.backbone.layer3(x2)
        x4 = self.backbone.layer4(x3)
        return x4

    def forward(self, x):
        x = self.cnn_feature_extractor(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(-1, self.emb_size)

        x = self.classifier(x)

        return x

In [ ]:
class CassvaClassifierV2(nn.Module):
    def __init__(self, n_classes: int = 5):
        super().__init__()
        self.model = timm.create_model('tf_efficientnet_b4_ns', pretrained=False)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, n_classes)

    def forward(self, x):
        return self.model(x)

In [ ]:
class CassvaClassifierV3(nn.Module):
    def __init__(self, n_classes: int = 5):
        super().__init__()
        self.model = timm.create_model('tf_efficientnet_b3_ns', pretrained=False)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, n_classes)

    def forward(self, x):
        return self.model(x)

# Inference Function

In [ ]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    for imgs in data_loader:
        image_preds = model(imgs.to(device).float())
        image_preds_all += [torch.softmax(image_preds, 1).cpu().numpy()]

    return np.concatenate(image_preds_all, axis=0)

# Load Models

In [ ]:
model_paths = sorted(glob(os.path.join('../input/leaf-disease-resnest50', '*')))
model_paths.extend(sorted(glob(os.path.join('../input/leaf-disease-effnet', '*'))))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def get_models(model_paths, recipe: str):
    models = []
    for model_path in model_paths:
        model_name: str = model_path.split('/')[-1]

        n_folds: int = int([x[-1] for x in model_name.split('-') if x.startswith('fold')][0])
        n_epochs: int = int([x[6:] for x in model_name.split('-') if x.startswith('epochs')][0])
            
        if not model_name.startswith(recipe):
            continue

        if model_name.startswith('resnest50_fast_4s2x40d-fmix-cutmix-'):
            model = CassvaClassifierV1().to(device)
        elif model_name.startswith('resnest50_fast_4s2x40d-cutmix-fmix-'):
            model = CassvaClassifierV1().to(device)
        elif model_name.startswith('resnest50_fast_4s2x40d-fcl-'):
            model = CassvaClassifierV1().to(device)
        elif model_name.startswith('effnetb4-fcl-'):
            model = CassvaClassifierV2().to(device)
        elif model_name.startswith('effnetb3-cutmix-scce-'):
            model = CassvaClassifierV3().to(device)
        elif model_name.startswith('effnetb4-cutmix-fmix-'):
            model = CassvaClassifierV2().to(device)
        elif model_name.startswith('effnetb4-pseudo-cutmix-fmix-'):
            model = CassvaClassifierV2().to(device)
        else:
            continue

        print(f'[+] load {model_name}')

        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()

        models.append(model)

    return models

# Load Test Data

In [ ]:
test = pd.DataFrame()
test['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))
test_ds = CassavaDataset(test, '../input/cassava-leaf-disease-classification/test_images/', transforms=get_inference_transforms())

tst_loader = torch.utils.data.DataLoader(
    test_ds, 
    batch_size=CFG['bs'],
    num_workers=CFG['num_workers'],
    shuffle=False,
    pin_memory=True
)

# Inference

In [ ]:
recipe_list = [
    # 'effnetb3-cutmix-scce-',
    'effnetb4-cutmix-fmix-',
    'effnetb4-fcl-',
    # 'effnetb4-pseudo-cutmix-fmix-',
    'resnest50_fast_4s2x40d-cutmix-fmix-',
    'resnest50_fast_4s2x40d-fcl-',
    'resnest50_fast_4s2x40d-fmix-cutmix-',
]

In [ ]:
preds = []
with torch.no_grad():
    for recipe in recipe_list:
        models = get_models(model_paths, recipe)

        preds_per_recipe = np.mean(
            [
                np.mean([inference_one_epoch(model, tst_loader, device) for _ in range(CFG['tta'])], axis=0)
                for model in models
            ], axis=0
        )

        preds.append(preds_per_recipe)

        del models

In [ ]:
# weights = [0.33547759, 0.30181755, 0.28914882]                          # Ensembles 9 weights
# weights = [0.32264375, 0.19517635, 0.10858799, 0.33353971]              # Ensembles 11 weights

# weights = [0.31301767, 0.29456512, 0.31728454]                          # Ensembles 20 weights
# weights_v1 = [0.21145703, 0.18271946, 0.26915887, 0.29597817]           # Ensembles 21 weights
# weights_v2 = [0.10884351, 0.2009535, 0.18187667, 0.20405396]            # Ensembles 21 weights - v3
# weights_v3 = [0.21411924, 0.18756664, 0.26507698, 0.31649887]           # Ensembles 21 weights - v4
# weights = [0.28008428, 0.08930099, 0.19287446, 0.13415098, 0.2855688]   # Ensembles 22 weights
# weights = [0.33566536, 0.10897427, 0.19606722, 0.31792425]              # Ensembles 23 weights
# weights = [0.22155271, 0.1881944, 0.38943474, 0.1644162]                # Ensembles 24 weigths - v1 (Simplex)
# weights = [0.29872177, 0.41167376, 0.92104135, 0.51346469]              # Ensembles 24 weights - v2 (Optuna)
weights_v1 = [0.16956903, 0.11774465, 0.3500565, 0.06345556, 0.27596693]   # Ensembles 25 weights - v1 (Simplex)
weights_v2 = [0.40763181, 0.23739473, 0.89755062, 0.15559366, 0.78777895]  # Ensembles 25 weights - v2 (Optuna)
weights_v3 = [0.213048, 0.1057116, 0.37792647, 0.10013445, 0.33992517]     # Ensembles 25 weights - v3 (Optuna) v1 wise

# weights = [0.34618164, 0.19092364, 0.38515934, 0.91232422, 0.00026995, 0.70023081]  # Ensembles 27 weights - v2 (Optuna)
# weights = [0.17911332, 0.10146231, 0.40657403, 0.30869513]              # Ensembles 28 weights - v1 (Simplex)
# weights = [0.13729324, 0.19570104, 0.81195183, 0.38014906]              # Ensembles 28 weights - v2 (Optuna)

# weights = [0.30978996, 0.22192819, 0.17375666, 0.29452519]              # Ensembles 21 weights w/ Optuna
# weights = [0.23779558, 0.24279284, 0.10746264, 0.41194895]              # Ensembles 21 weights + 2020 validation w/ Optuna
# weights = [0.22772560, 0.10498674, 0.19491591, 0.18363636, 0.28873539]  # Ensembles 22 weights w/ Optuna
# weights = [0.22084675, 0.08012831, 0.24442862, 0.07493153, 0.37966478]  # Ensembles 22 weights + 2020 validation w/ Optuna

# weights = [0.31792425, 0.33566536, 0.19606722, 0.10897427]              # Ensembles 21 v1-corr
# weights = [0.90971052, 0.82730546, 0.38318065, 0.26230337]              # Ensembles 21 v2-corr
# weights = [0.16892105, 0.13211221, 0.54735528, 0.06826233, 0.06961585]  # Ensembles 25 v1-corr
# weights = [0.3306592, 0.25242486, 0.8238158, 0.11380859, 0.22893606]    # Ensembles 25 v2-corr
# weights = [0.64438387, 0.06787352, 0.21374317, 0.92894338, 0.30073056, 0.25681572]  # Ensembles 27 v2-corr

weights = [round((x + y + z) / 3., 4) for x, y, z in zip(weights_v1, weights_v2, weights_v3)]

In [ ]:
# weights = [0.16127426, 0.14639634, 0.40610428, 0.25388546]                          # Ensembles 29 v1
# weights = [0.14392024, 0.26999754, 0.66471911, 0.10123768]                          # Ensembles 29 v2
# weights = [0.62811276, 0.23633465, 0.25867451, 0.95243224, 0.10789748, 0.5444512]   # Ensembles 30 v2
# weights = [0.15856572, 0.03564624, 0.09446308, 0.06490467, 0.51857468, 0.12564358]  # Ensembles 31 v1
# weights = [0.37460579, 0.06460235, 0.06938154, 0.22316517, 0.83760474, 0.26799389]  # Ensembles 31 v2
# weights = [0.03564624, 0.15856572, 0.12564358, 0.51857468, 0.06490467, 0.09446308]  # Ensembles 32 v1
# weights = [2.47e-05, 0.58436138, 0.5120863, 0.87538525, 0.61622132, 0.31402191]     # Ensembles 32 v2
# weights = [0.50718439, 0.16996559, 0.25776149, 0.13448321, 0.92798265, 0.2179295, 0.22300932]  # Ensembles 33 v2
# weights = [0.72592935, 0.90095763, 0.65667935, 0.53306918]                          # Ensembles 34 v2
# weights = [0.29556101, 0.95351582, 0.96590418, 0.82782731, 0.73989029]              # Ensembles 35 v2
# weights = [0.24627387, 0.12512951, 0.18096359, 0.07093961, 0.37141577]              # Ensembles 35 v3

weights

In [ ]:
# tst_preds = np.mean(preds, axis=0)

# tst_preds = preds[0]
# tst_preds = weights[0] * preds[0] + weights[1] * preds[1] + weights[2] * preds[2]
# tst_preds = weights[0] * preds[0] + weights[1] * preds[1] + weights[2] * preds[2] + weights[3] * preds[3]
tst_preds = weights[0] * preds[0] + weights[1] * preds[1] + weights[2] * preds[2] + weights[3] * preds[3] + weights[4] * preds[4]
# tst_preds = weights[0] * preds[0] + weights[1] * preds[1] + weights[2] * preds[2] + weights[3] * preds[3] + weights[4] * preds[4] + weights[5] * preds[5]
# tst_preds = weights[0] * preds[0] + weights[1] * preds[1] + weights[2] * preds[2] + weights[3] * preds[3] + weights[4] * preds[4] + weights[5] * preds[5] + weights[6] * preds[6]

# Submission

In [ ]:
test['label'] = np.argmax(tst_preds, axis=1)

In [ ]:
test.to_csv('submission.csv', index=False)
test

# EOF